# Import Libraries

In [1]:
from swarmintelligence import MemoryBasedGreyWolfOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools
import math
import random as rd

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 1))

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/memGWO/optimal_params.pkl', 'rb'))
optimal_params

{'wolfSize': 25, 'maxIteration': 100, 'crossover_rate': 1.0}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['mMasi_entropy']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo = MemoryBasedGreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        crossover_rate=optimal_params['crossover_rate'],
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs, best_thresholds = gwo.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(gwo.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(gwo.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                gwo_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions'].append(mean_regions)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['Fitness'].append(fitness_values)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7106129145502
1948.7183902321062
1948.7106129145502
1948.7183902321062
1948.7183902321062
Mean Fitness = 1948.7178717442687
	 AirplaneF16.tiff | Threshold =  3
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.805475833105
2024.805475833105
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8020017977378
2024.8375098105803
2024.8375098105803
2024.805475833105
2024.8375098105803
2024.8375098105803
2024.8020017977378
2024

3254.604648922451
3254.60318711213
3254.472575716386
3254.518914349312
3254.5660832024537
Mean Fitness = 3254.477328848731
	 Mandrill.tiff | Threshold =  2
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
Mean Fitness = 1549.02713277089
	 Mandrill.tiff | Threshold =  3
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.4542198585186
1639.3827918335382
1639.466171099567
1639.466171099567
1639.466171099567
1639.4542

4217.821298877458
4218.034079186658
4217.937222685617
4218.026068576767
4218.037279885386
4218.037279885386
4218.034079186658
4217.94772962205
4218.037279885386
4218.034079186658
4217.990380722911
Mean Fitness = 4217.9880020261


## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,mMasi_entropy,max,"[113, 173]",1.137710,1948.717872,2269.814377,47.642558,14.570904,0.794437,0.940232,"[[113, 255, 255, 255, 255, 255, 255, 255, 255,...","[0.8949089050292969, 0.9719548225402832, 1.126...","[1948.7183902321062, 1948.7183902321062, 1948....","[2270.6405181884766, 2270.6405181884766, 2270....","[47.651238369936166, 47.651238369936166, 47.65...","[14.569319775617668, 14.569319775617668, 14.56...","[0.7944112258870609, 0.7944112258870609, 0.794...","[0.9402201034707982, 0.9402201034707982, 0.940..."
1,AirplaneF16.tiff,3,mMasi_entropy,max,"[93, 145, 191]",1.086729,2024.827436,1769.594390,42.066282,15.652176,0.778377,0.953308,"[[93, 255, 255, 191, 191, 191, 255, 191, 255, ...","[1.2290565967559814, 1.157872200012207, 1.1967...","[2024.8375098105803, 2024.8375098105803, 2024....","[1774.3395538330078, 1774.3395538330078, 1774....","[42.12291008267363, 42.12291008267363, 42.1229...","[15.640436268576428, 15.640436268576428, 15.64...","[0.77934507049225, 0.77934507049225, 0.7793450...","[0.9532349424759257, 0.9532349424759257, 0.953..."
2,AirplaneF16.tiff,4,mMasi_entropy,max,"[84, 129, 172, 203]",0.955852,2070.053352,1092.587863,33.053929,17.746445,0.732604,0.969380,"[[84, 203, 203, 203, 203, 203, 203, 203, 203, ...","[0.830437421798706, 1.0873453617095947, 0.8872...","[2070.0794736113085, 2069.8648340198806, 2070....","[1094.045597076416, 1093.1258392333984, 1094.0...","[33.07636009412789, 33.06245361786385, 33.0763...","[17.74044938188586, 17.744102006209904, 17.740...","[0.7326687134455045, 0.7314337006877252, 0.732...","[0.9693723689797689, 0.9696124663565749, 0.969..."
3,AirplaneF16.tiff,5,mMasi_entropy,max,"[70, 108, 143, 180, 205]",0.840863,2095.879393,906.575271,30.108332,18.557375,0.745369,0.975414,"[[70, 205, 205, 205, 180, 205, 205, 205, 205, ...","[0.8176400661468506, 1.007716178894043, 0.7717...","[2096.0455757494956, 2096.1432770275533, 2095....","[909.3038101196289, 907.2876815795898, 901.879...","[30.154664815242583, 30.121216469120064, 30.03...","[18.54371350017661, 18.55335346450818, 18.5793...","[0.7450461077231622, 0.7452811482856196, 0.745...","[0.9753498161289982, 0.9753724479476479, 0.975..."
4,Lena.png,2,mMasi_entropy,max,"[93, 151]",0.769805,1961.822013,2813.567070,53.043068,13.638231,0.612182,0.883309,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[0.7672407627105713, 0.8597414493560791, 0.811...","[1961.8220125153634, 1961.8220125153634, 1961....","[2813.567070007324, 2813.567070007324, 2813.56...","[53.04306806744237, 53.04306806744237, 53.0430...","[13.638230885124642, 13.638230885124642, 13.63...","[0.6121817460412936, 0.6121817460412936, 0.612...","[0.8833090574580458, 0.8833090574580458, 0.883..."
5,Lena.png,3,mMasi_entropy,max,"[81, 127, 171]",0.809106,2128.297282,1250.481210,35.362137,17.160035,0.706366,0.933683,"[[171, 171, 171, 171, 171, 171, 171, 171, 171,...","[1.0184009075164795, 0.7104368209838867, 1.165...","[2128.284181016679, 2128.3048672466566, 2128.2...","[1252.4491691589355, 1249.3418655395508, 1252....","[35.389958592218434, 35.346030407098766, 35.38...","[17.153202518789254, 17.16399067408044, 17.153...","[0.7072313681226016, 0.7058656377663244, 0.707...","[0.9340987864621145, 0.9334425202424386, 0.934..."
6,Lena.png,4,mMasi_entropy,max,"[75, 114, 145, 180]",0.766039,2191.860405,794.692550,28.190266,19.128828,0.753547,0.954174,"[[180, 180, 180, 180, 180, 180, 180, 180, 180,...","[0.7597589492797852, 0.6845195293426514, 0.773...","[2191.8742974576116, 2191.8742974576116, 2191....","[794.4945793151855, 794.4945793151855, 794.494...","[28.186780222565073, 28.186780222565073, 28.18...","[19.129894224227296, 19.129894224227296, 19.12...","[0.7535416904705244, 0.7535416904705244, 0.753...","[0.9541814904633167, 0.9541814904633167, 0.954..."
7,Lena

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/memGWO/memGWO_masi_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/memGWO/memGWO_masi_dict.pkl', 'wb'))